# 텍스트 이진분류 (영어 IMDB)

In [0]:
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# 경로 설정
imdb_dir = '/content/gdrive/My Drive/pytest/aclImdb_v1_small/aclImdb/'

!ls '/content/gdrive/My Drive/pytest/aclImdb_v1_small/aclImdb/'

In [0]:
# Train Data Loading
import os
train_dir = os.path.join(imdb_dir, 'train')
labels = [] ; texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname), encoding='utf8')
      texts.append(f.read())
      f.close()
      if label_type == 'neg':
        labels.append(0)
      else:
        labels.append(1)

In [0]:
# Data 확인
print('texts 0:', texts[0])
print('texts len:', len(texts))

print('labels 0:', labels[0])
print('labels len:', len(labels))

print('texts type:', type(texts))

In [0]:
# Tokenizer 연습
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 10000

sample1 = [['사과 감자 옥수수'], ['딸기 감자 옥수수'], ['양파 감자 옥수수'], ['양파 부추 옥수수']]
sample2 = [['사과', '감자', '옥수수'], ['너희', '감자', '옥수수'], ['그들', '감자', '옥수수'], ['양파', '부추', '옥수수']]
sample3 = ['사과 감자 옥수수 너희 그들 양파 부추']
sample4 = ['사과', '감자', '옥수수', '너희', '그들', '양파', '부추']

tokenizer = Tokenizer(num_words=max_words)	      # Tokenizer 객체 생성
tokenizer.fit_on_texts(sample1)     			        # 단어 인덱스를 구축한다 
word_index = tokenizer.word_index           		  # 단어 인덱스만 가져온다
print(word_index)

tokenizer = Tokenizer(num_words=max_words)	      # Tokenizer 객체 생성
tokenizer.fit_on_texts(sample2)     			        
word_index = tokenizer.word_index           		  
print(word_index)

tokenizer = Tokenizer(num_words=max_words)	      # Tokenizer 객체 생성
tokenizer.fit_on_texts(sample3)     			         
word_index = tokenizer.word_index           		  
print(word_index)

tokenizer = Tokenizer(num_words=max_words)	      # Tokenizer 객체 생성
tokenizer.fit_on_texts(sample4)     			         
word_index = tokenizer.word_index           		  
print(word_index)

In [0]:
# Data Tokenizing
# 텍스트에 사용된 단어의 종류를 빈도 순으로 정렬하는 작업을 수행한다
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

validation_ratio = math.floor(len(texts) * 0.3)  	# 검증 샘플은 전체의 30%로 한다
max_words = 10000               			            # 데이터셋에서 가장 빈도 높은 10,000 개의 단어만 사용한다
maxlen = 200					                            # 항상 200 단어가 되도록 길이를 고정한다

tokenizer = Tokenizer(num_words=max_words)	      # 상위빈도 10,000 개의 단어만을 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(texts)     			          # 단어 인덱스를 구축한다 
word_index = tokenizer.word_index           		  # 단어 인덱스만 가져온다 

In [0]:
# Tokenizing 결과 확인
print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ', word_index)

In [0]:
# Data Sequencing
# 문자를 숫자로 변환하는 작업을 수행한다
# 상위 빈도 10,000(max_words)개의 단어만 추출하여 word_index의 숫자 리스트로 변환한다.
data = tokenizer.texts_to_sequences(texts)		# Tokenizer 결과가 여기서 반영된다.

print('data 0:', data[0])
print('texts 0:', texts[0])				           # texts[0]의 본래 단어들

In [0]:
print(type(texts))
print(type(data))

In [0]:
# Data Pading 연습
from keras.preprocessing.sequence import pad_sequences

sequences = [[1, 2, 3, 4, 5], [1, 2, 3, 4], [1]]		  # nested list
padded = pad_sequences(sequences, maxlen=3)		  # 2D tensor 
print(padded)

In [0]:
# Data Pading
data = pad_sequences(data, maxlen=maxlen) 

print('data:', data)
print('data 0:', data[0])
print(len(data[0]))

print(word_index)

In [0]:
print(type(texts))
print(type(data))
print(data.shape)

In [0]:
# One-Hot-Encoding 연습
sample = [[5, 6, 7], [8, 9, 10]]
arr = np.zeros((len(sample), 10+1))		# “10”은 11번째에 들어가게 되므로 11개의 공간을 만들어야 한다
for i, seq in enumerate(sample):		  # 리스트가 2개이므로 i는 총 2회(0, 1) 반복되며,
   	arr[i, seq] = 1.				          # 각 i에서 리스트의 number가 가리키는 곳에 1을 기록한다
arr

In [0]:
# One-Hot-Encoding
def to_one_hot(sequences, dimension):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results


data = to_one_hot(data, dimension=max_words) 
labels = np.asarray(labels).astype('float32')   

In [0]:
# One-Hot-Encoding 결과 확인
print('data:', data)
print(len(data[0]))					# dimension=10000으로 했으므로 각 행은 10,000개를 가지고 있다
print('data [0][0:100]:', data[0][0:100])

print(word_index)

In [0]:
print(type(texts))
print(type(data))
print(data.shape)

In [0]:
# Train 데이터와 Validation 데이터 준비

print('데이터 텐서의 크기:', data.shape)  		# (25000, 10000)
print('레이블 텐서의 크기:', labels.shape) 		# (25000,) data와 label이 모두 2D 텐서가 되었음

indices = np.arange(data.shape[0]) 		        # 0 ~ 24999 까지의 숫자를 생성
np.random.shuffle(indices)     			          # 0 ~ 24999 까지의 숫자를 랜덤하게 섞음
data = data[indices]    				              # 이것을 인덱스로 하여 2D 텐서 데이터를 섞음 
labels = labels[indices]				              # label도 같은 순서로 섞음 

print(indices)

In [0]:
# 훈련데이터와 검증데이터 분리
x_train = data[validation_ratio:] 			      # 훈련데이터의 70%를 훈련데이터
y_train = labels[validation_ratio:] 			    # 훈련데이터의 70%를 훈련데이터 Label (data와 labels는 같은 순서)
x_val = data[:validation_ratio] 			        # 훈련데이터의 30%를 검증데이터
y_val = labels[:validation_ratio] 			      # 훈련데이터의 30%를 검증데이터 Label

In [0]:
# 모델 정의하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()                                              # 모델을 새로 정의

model.add(Dense(64, activation='relu', input_shape=(max_words,)))	# 첫 번째 은닉층
model.add(Dense(32, activation='relu'))                           # 두 번째 은닉층
model.add(Dense(1, activation='sigmoid'))                 		    # 출력층

In [0]:
# 모델 요약 출력
model.summary()

In [0]:
# Compile & Train Model
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

# 모델 훈련
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
history_dict = history.history

In [0]:
# 경로 변경
%cd /content/gdrive/My Drive/pytest/

In [0]:
# Save Model
# multidimensional numpy arrays를 저장할 수 있는 h5 file(HDF) 포맷으로 저장한다
model.save('text_binary_model.h5')


# 훈련데이터에서 사용된 상위빈도 10,000개의 단어로 된 Tokenizer 저장
# 새로 입력되는 문장에서도 같은 단어가 추출되게 한다
import pickle
with open('text_binary_tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Accuracy & Loss 확인
# history 딕셔너리 안에 있는 정확도와 손실값을 가져와 본다

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Accuracy of each epoch:', acc)		# [0.79, 0.90, 0.93, 0.94, 0.96, 0.97, 0.98, 0.98, 0.98, 0.99]
epochs = range(1, len(acc) +1)			# range(1, 11)

In [0]:
# Plotting Accuracy
import matplotlib.pyplot as plt

# 훈련데이터의 정확도에 비해 검증데이터의 정확도는 낮게 나타난다
# epoch가 높아지면 모델은 훈련데이터에 매우 민감해져 오히려 새로운 데이터를 잘 못 맞춘다
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.legend()

In [0]:
# Plotting Loss
plt.figure()    # 새로운 그림을 그린다

# 훈련데이터의 손실값은 낮아지나, 검증데이터의 손실값은 높아진다
# 손실값은 오류값을 말한다. 예측과 정답의 차이를 거리 계산으로 구한 값이다
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
# Load Model
import os
from tensorflow.keras.models import load_model

filepath = '/content/gdrive/My Drive/pytest/'
os.chdir(filepath)
print("Current Directory:", os.getcwd())

loaded_model = load_model('text_binary_model.h5')
print("model loaded:", loaded_model)

with open('text_binary_tokenizer.pickle', 'rb') as handle:
  loaded_tokenizer = pickle.load(handle)

In [0]:
# Test Data Loading
test_dir = os.path.join(imdb_dir, 'test')
labels = [] ; texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(test_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname), encoding='utf8')
      texts.append(f.read())
      f.close()
      if label_type == 'neg':
        labels.append(0)
      else:
        labels.append(1)

In [0]:
# Data 확인
print('texts:', texts[0])
print('texts len:', len(texts))

In [0]:
# Data Sequencing
# 문자열을 word_index의 숫자 리스트로 변환
data = loaded_tokenizer.texts_to_sequences(texts)

# padding으로 문자열의 길이를 고정시킨다
data = pad_sequences(data, maxlen=maxlen) 

# test 데이터를 원-핫 인코딩한다
x_test = to_one_hot(data, dimension=max_words)

# label을 list에서 넘파이 배열로 변환. 결과가 0 또는 1만 나오므로 이와같이 int32로 저장해도 된다.
y_test = np.asarray(labels)

In [0]:
# Test Data Evaluation
test_eval = loaded_model.evaluate(x_test, y_test)	  # 모델에 분류할 데이터와 그 정답을 같이 넣어준다
print('prediction model loss & acc:', test_eval)		# 모델이 분류한 결과와 입력된 정답을 비교한 결과

In [0]:
# 1개 데이터 예측
text = ["Hi, this is a test sentence."]      # 데이터를 list 타입으로 만든다
data = loaded_tokenizer.texts_to_sequences(text)
data = pad_sequences(data, maxlen=maxlen)
x_test = to_one_hot(data, dimension=max_words)

prediction = loaded_model.predict(x_test)
print("Result:", prediction)					       # [[0.53135556]]. 1이 될 확률이 53.1%